In [ ]:
!pip install datasets peft spacy coreferee

In [3]:
from transformers import pipelines, PegasusForConditionalGeneration, PegasusTokenizer, DataCollatorForSeq2Seq, AutoTokenizer, AutoModelForSeq2SeqLM, Trainer, TrainingArguments
import transformers
import pandas as pd
from datasets import Dataset, load_from_disk
import json
from peft import get_peft_model, LoraConfig, TaskType, PeftModel

In [ ]:
# transformers.logging.set_verbosity_info()

In [ ]:
# Load model directly

# tokenizer = AutoTokenizer.from_pretrained("google/pegasus-large")
# model = AutoModelForSeq2SeqLM.from_pretrained("google/pegasus-large")

# data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model)

loading weights file pytorch_model.bin from cache at /root/.cache/huggingface/hub/models--google--pegasus-large/snapshots/dec7796b22f29b7d1c476192313eae8ed57b6b77/pytorch_model.bin
Generate config GenerationConfig {
  "bos_token_id": 0,
  "decoder_start_token_id": 0,
  "eos_token_id": 1,
  "forced_eos_token_id": 1,
  "length_penalty": 0.8,
  "max_length": 256,
  "num_beams": 8,
  "pad_token_id": 0
}

All model checkpoint weights were used when initializing PegasusForConditionalGeneration.

Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-large and are newly initialized: ['model.decoder.embed_positions.weight', 'model.encoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


generation_config.json:   0%|          | 0.00/260 [00:00<?, ?B/s]

loading configuration file generation_config.json from cache at /root/.cache/huggingface/hub/models--google--pegasus-large/snapshots/dec7796b22f29b7d1c476192313eae8ed57b6b77/generation_config.json
Generate config GenerationConfig {
  "bos_token_id": 0,
  "decoder_start_token_id": 0,
  "eos_token_id": 1,
  "forced_eos_token_id": 1,
  "length_penalty": 0.8,
  "max_length": 256,
  "num_beams": 8,
  "pad_token_id": 0
}



In [ ]:
# from datasets import load_from_disk

# # Load the dataset from disk
# tokenized_dataset = load_from_disk("/content/drive/MyDrive/Data/tokenized_subset_cnn_dailymail")

In [ ]:
# split_dataset = tokenized_dataset.train_test_split(test_size=0.1)

# train_dataset = split_dataset['train']
# val_dataset = split_dataset['test']
# tokenized_train_dataset = train_dataset.remove_columns(['article', 'highlights', 'id'])
# tokenized_val_dataset = val_dataset.remove_columns(['article', 'highlights', 'id'])

# print(f"Train size: {len(tokenized_train_dataset)}")
# print(tokenized_train_dataset.column_names)
# print(f"Validation size: {len(tokenized_val_dataset)}")

Train size: 90000
['input_ids', 'attention_mask', 'labels']
Validation size: 10000


In [ ]:
# lora_config = LoraConfig(
#     task_type=TaskType.SEQ_2_SEQ_LM,  # Sequence-to-sequence task
#     r=8,  # Rank of the low-rank adaptation matrices
#     lora_alpha=32,  # Scaling factor for LoRA layers
#     lora_dropout=0.1,  # Dropout rate for LoRA layers
#     target_modules=["q_proj", "v_proj"],  # Target modules to apply LoRA (query and value projections)
# )

# lora_model = get_peft_model(model, lora_config)

# training_args = TrainingArguments(
#     output_dir='./results',
#     per_device_train_batch_size=4,
#     per_device_eval_batch_size=4,
#     learning_rate=1e-4,
#     num_train_epochs=3,
#     evaluation_strategy='epoch',
#     save_strategy="epoch",
#     logging_steps=100,
#     fp16=True,  # Enable mixed precision if using a compatible GPU
#     report_to="none",  # Disable unnecessary logging to external systems like WandB
# )

In [ ]:
# trainer = Trainer(
#     model=lora_model,
#     args=training_args,
#     train_dataset=tokenized_train_dataset,
#     eval_dataset=tokenized_val_dataset,
#     data_collator = data_collator
# )

In [ ]:
# trainer.train()

In [ ]:
base_model = PegasusForConditionalGeneration.from_pretrained("google/pegasus-large")

lora_model = PeftModel.from_pretrained(base_model, "/content/drive/MyDrive/Data/OP/lora-pegasus")

In [ ]:
# with open('/content/drive/MyDrive/Data/OP/training_logs.json', 'r') as f:
#     log_history = json.load(f)

In [ ]:
# log_history

In [ ]:
# import matplotlib.pyplot as plt

# epochs = [log['epoch'] for log in log_history if 'loss' in log]
# losses = [log['loss'] for log in log_history if 'loss' in log]

# # Plot the loss over epochs
# plt.figure(figsize=(8, 6))
# plt.plot(epochs, losses, label='Training Loss')
# plt.xlabel('Epoch')
# plt.ylabel('Loss')
# plt.title('Training Loss Over Epochs')
# plt.legend()
# plt.savefig('training_loss_plot.png')  # Save the plot as an image
# plt.show()

In [ ]:
tokenizer = PegasusTokenizer.from_pretrained('google/pegasus-large')

In [41]:
input_text = """
LAS VEGAS, Nevada (CNN) -- Former football star O.J. Simpson will be held without bail after his arrest on robbery and assault charges, police announced late Sunday. Police released this mug shot of O.J. Simpson after his arrest. Simpson is accused of having directed several other men in an alleged armed robbery of sports memorabilia in a room at a Las Vegas hotel room. Las Vegas authorities said they have no information leading them to believe Simpson was carrying a firearm during the alleged incident at the Palace Station Hotel and Casino. Police said Simpson and other men burst into the room and walked out with the memorabilia, including some that was unrelated to Simpson, police said. "We don't believe that anyone was roughed up, but there were firearms involved," Lt. Clint Nichols told reporters. Nichols said the firearms were pointed at the victims. A reporter asked Nichols: Was "O.J. was the boss in that room?" Nichols responded, "That is what we believe, yes." Watch Simpson transferred Sunday in handcuffs » . The alleged victims were identified as Bruce Fromong, a sports memorabilia collector who described the incident as "a home invasion-type robbery," and Alfred Beardsley, who has been quoted by celebrity Web site TMZ.com as saying that Simpson later apologized to him and told him he regretted the incident. Acting on a tip, police met over the weekend at McCarran International Airport with 46-year-old Walter Alexander, of Mesa, Arizona, who told them about the alleged robbery and validated the tipster's information, Capt. James Dillon told reporters. Alexander was arrested Saturday night on two counts of robbery with a deadly weapon, two counts of assault with a deadly weapon, conspiracy to commit robbery and burglary with a deadly weapon. He was released on his own recognizance and returned to Mesa either Saturday night or early Sunday morning, Dillon said. In addition, investigators are seeking four other men they believe accompanied Simpson into the hotel room, Nichols said. Nichols said, "There is a social relationship between the individuals that we identified and O.J. Simpson." Though Simpson is not accused of having brandished a gun himself, two firearms that police said were used were recovered early Sunday in one of three searches. Investigators would would not divulge where the weapons were found. Nichols dismissed an initial report that the men may have been off-duty police. "There is no truth to that whatsoever," he said. "That came as a result of some language that was used when the individuals burst into the room that led our victims to believe that they may have been police." Simpson, 60, has acknowledged taking some items that belonged to him, but he has denied that any weapons were involved. "Whether the property belonged to Mr. Simpson or not is still in debate," Nichols said. "We are still in the process of sorting that out." Nichols also said that some of the property taken had Simpson's signature. But "there was some other property taken as well," he said. "I believe there were some Joe Montana cleats and some signed baseballs and other stuff." The latest charges against Simpson mean he faces the prospect of another prosecution, more than a decade after the June 1994 stabbing deaths of his ex-wife, Nicole Brown Simpson, and Ron Goldman. Simpson was acquitted of murder the following year. The trial riveted much of the United States. But in 1997, a jury found him liable for the deaths in a civil case brought by the Goldman family. Simpson was ordered to pay the families a total of $33.5 million for the deaths . Goldman had gone to Nicole Simpson's Los Angeles home to return a pair of glasses the day of the slayings. Goldman's sister, Kim Goldman, said she wasn't surprised by the robbery allegations, since Simpson "thinks he can do no wrong." "He's capable of stabbing people to death, so I think robbery is nothing surprising," she said. "Normal, logical, civil-minded, law-abiding people don't storm a room with guns demanding stuff back." Fromong had testified on Simpson's behalf in the civil case, telling the court that prices for Simpson memorabilia had dropped substantially since the 1995 verdict. His testimony was part of the defense's contention that Simpson could not afford to pay the Goldmans. Simpson recently wrote a book originally titled "If I Did It" and had planned to publish it himself, but a public outcry led to the cancellation of his book deal. A bankruptcy judge subsequently awarded the Goldmans the rights to the book in light of their inability to collect the wrongful death award. The Goldmans retitled the book, "If I Did It: Confessions of the Killer." That book just hit bookstores. E-mail to a friend . CNN's Ted Rowlands contributed to this report.
"""

input_ids = tokenizer(
    input_text,
    return_tensors="pt",  # Return tensors for PyTorch
    padding="longest",
    truncation=True,
    max_length=512  # Adjust this as needed based on input size
).input_ids

In [42]:
# Generate the summary or text
output_ids = lora_model.base_model.generate(
    input_ids,
    max_length=128,
    num_beams=5,
    early_stopping=True
)

# Decode the generated output
generated_text = tokenizer.decode(output_ids[0], skip_special_tokens=True)

print("Generated Text:", generated_text)


Generated Text: O.J. Simpson will be held without bail after his arrest on robbery and assault charges, police say. Police say they have no information leading them to believe Simpson was carrying a firearm during the alleged incident. Walter Alexander, 46, of Mesa, Arizona, was arrested Saturday night, police say.


In [ ]:
# !python -m spacy download en_core_web_md

In [34]:
# !python -m coreferee install en

In [44]:
import spacy
import coreferee

# Initialize spaCy model and coreferee
nlp = spacy.load('en_core_web_md')
nlp.add_pipe('coreferee')

# Process the text to resolve coreferences
doc = nlp(text)

doc._.coref_chains.print()

0: Simpson(1), his(8), Simpson(29)
1: police(16), them(26)
2: Police(19), they(21)
